<a href="https://colab.research.google.com/github/sodi16/Suspicious-Behavior-Recognition/blob/main/Suspicious_Behavior_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import shutil
import copy
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import sys
from google.colab.patches import cv2_imshow
from sklearn.metrics import recall_score, classification_report
import pylab as pl
import tensorflow as tf
from tensorflow.python.keras.layers import Input, Dense, ConvLSTM2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.python.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from PIL import Image

os.chdir('/content/drive/My Drive/data_caviar')

Mounted at /content/drive


In [ ]:
#if model already trained
model = load_model('model4.h5')
with open('fight_new.npy', 'rb') as f:
  X = np.load(f, allow_pickle=True)
f.close()

#if data already saved
with open(r'all_label/X.npy', 'rb') as f:
  X = np.load(f, allow_pickle=True)
f.close()
with open(r'all_label/y.npy', 'rb') as f:
  y = np.load(f, allow_pickle=True)
f.close()
with open(r'all_label/X_test.npy', 'rb') as f:
  X_test = np.load(f, allow_pickle=True)
f.close()
with open(r'all_label/y_test.npy', 'rb') as f:
  y_test = np.load(f, allow_pickle=True)
f.close()

In [2]:
all_roles = {'fighters': 2, 'fighter': 2, 'leaving object': 2, 'browser': 1, 'browsers': 1, 'walkers': 0, 'meet': 0, 'meeters': 0, 'walker': 0}
all_context = {'fighting': 2, 'leaving': 2, 'drop down': 2, 'browsing': 1, 'immobile': 0, 'walking': 0, 'meeting': 0, 'windowshop': 0, 'shop enter': 0, 'shop exit': 0, 'shop reenter': 0, 'none':0}

def display_all_suspicious_images(images, labels):
    img = None
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(len(images)):
        image = copy.copy(images[i,:,:,:])
        label = labels[i,:]
        im_shape = image.shape
        if label >= 0.5:
            image = cv2.rectangle(image, (1, 1), (im_shape[1]-1, im_shape[0]-1) , [255, 0, 0], 3)
            image = cv2.putText(img=image,text='Suspicious Behavior', org=(12,12), fontFace=font ,fontScale=0.4, color=[255,0,0], lineType=3)
        else:
            image = cv2.putText(img=image,text='Not Suspicious Behavior', org=(12,12), fontFace=font ,fontScale=0.4, color=[255,0,0], lineType=3)
        if img is None:
            img = pl.imshow(image)
        img.set_data(image)
        pl.draw()
        pl.title('Frame ' + str(i))

        if id == 1000:
            break
            pl.close('all')

        if cv2.waitKey(0) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            pl.close('all')
            break


def suspicious_behavior_labels(labels):
    very_suspicious_index = 2
    suspicious_index = 1
    not_suspicious_index = 0
    num_frames = len(labels)

    new_labels = np.array([],dtype=int)
    check_role = np.array([])
    check_context = np.array([])

    for i in range(num_frames):
        check_role = np.append(check_role, np.max())
        check_context = np.append(check_context, np.min([switcher.get(item) for item in labels[i, 1]]))

    for i in range(len(labels)):
        if check_role[i] in very_suspicious_index  or check_context[i] in very_suspicious_index:
            new_labels = np.append(new_labels, 2)
        elif check_role[i] in suspicious_index  or check_context[i] in suspicious_index:
            new_labels = np.append(new_labels, 1)
        else:
            new_labels = np.append(new_labels, 0)

    return new_labels.reshape(len(new_labels), 1)


def plot_history(history):
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

In [ ]:
################### Resnet50 TRANSFER MODEL
video = Input(shape=(40, 224, 224, 3))
cnn_base = ResNet50(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
cnn_out = GlobalAveragePooling2D()(cnn_base.output)
cnn = Model(inputs=cnn_base.input, outputs=cnn_out)
cnn.trainable = False
x = TimeDistributed(cnn)(video)
x = GRU(512, return_sequences=True)(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model([video], x)
model.summary()

earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["accuracy"], )
model.fit(X, y, batch_size=20, epochs=100, shuffle=False, callbacks=callbacks)


In [ ]:
model.save('model4.h5', overwrite=True)

In [3]:
############################################### NEW DATA IN GRAY CHANNEL
os.chdir('/content/drive/My Drive/data_caviar')
with open(r'label_Walk/X.npy', 'rb') as f:
  X_walk = np.load(f, allow_pickle=True)
f.close()
with open(r'label_Walk/y.npy', 'rb') as f:
  y_walk = np.load(f, allow_pickle=True)
f.close()

with open(r'label_Fight/X.npy', 'rb') as f:
  X_fight = np.load(f, allow_pickle=True)
f.close()
with open(r'label_Fight/y.npy', 'rb') as f:
  y_fight = np.load(f, allow_pickle=True)
f.close()

with open(r'label_LeftBag/X.npy', 'rb') as f:
  X_leftbag = np.load(f, allow_pickle=True)
f.close()
with open(r'label_LeftBag/y.npy', 'rb') as f:
  y_leftbag = np.load(f, allow_pickle=True)
f.close()

with open(r'label_Groups_Meeting/X.npy', 'rb') as f:
  X_groups = np.load(f, allow_pickle=True)
f.close()
with open(r'label_Groups_Meeting/y.npy', 'rb') as f:
  y_groups = np.load(f, allow_pickle=True)
f.close()

with open(r'label_Rest/X.npy', 'rb') as f:
  X_rest = np.load(f, allow_pickle=True)
f.close()
with open(r'label_Rest/y.npy', 'rb') as f:
  y_rest = np.load(f, allow_pickle=True)
f.close()

gen = ImageDataGenerator()
new_fight = []
for i in X_fight[:]:
  new_fight.append(gen.apply_transform(i, transform_parameters={'theta':10, 'brightness':0.85, 'flip_horizontal':True, 'shear_range':0.15,'zoom_range':0.15}))
new_fight = np.array(new_fight)

gen = ImageDataGenerator()
new_leftbag = []
for i in X_leftbag[:]:
  new_leftbag.append(gen.apply_transform(i, transform_parameters={'theta':10,'brightness':0.85,'flip_horizontal':True, 'shear_range':0.15,'zoom_range':0.15}))
new_leftbag = np.array(new_leftbag)

gen = ImageDataGenerator()
new_groups = []
for i in X_groups[:]:
  new_groups.append(gen.apply_transform(i, transform_parameters={'theta':10,'brightness':0.85, 'flip_horizontal':True, 'shear_range':0.15, 'zoom_range':0.15}))
new_groups = np.array(new_groups)


In [ ]:
X = np.concatenate((X_walk[:-400], X_groups[:-200], new_groups[300:], X_fight[:-200], new_fight[:-100], X_leftbag[:-500], new_leftbag[:-400], X_rest[:-249]),axis=0)
y = np.concatenate((y_walk[:-400], y_groups[:-200], y_groups[:-300], y_fight[:-200], y_fight[:-100], y_leftbag[:-500], y_leftbag[:-400], y_rest[:-249]),axis=0)

X = X[:,10:285,30:335]
X = np.array([cv2.resize(i, (224,224)) for i in X[:,:]])

#X = np.array([cv2.cvtColor(i, cv2.COLOR_BGR2GRAY) for i in X[:]])   # TO GREYSCALE
#X = X.reshape(30, 90, 224, 224, 1)       # TO GREYSCALE

X = np.array([i for idx,i in enumerate(X[:]) if idx %3 != 0])
y = np.array([i for idx,i in enumerate(y[:]) if idx %3 != 0])

X = X[8:]
y = y[8:]

#For 1 classification
y[y == 2] = 1  

############## One hot encoding for 3 class
"""
onehot_encoder = OneHotEncoder(sparse=False)
y = onehot_encoder.fit_transform(y)
y = y.astype(np.int8)
y = y.reshape(88, 40, 3)
X = X.reshape(88, 40 ,224, 224, 3)"""

X = X.reshape(98, 40 ,224, 224, 3)
y = y.reshape(98, 40, 1)

#del X_fight, X_groups, X_leftbag, X_rest, X_walk, new_fight, new_leftbag, new_groups
#del y_fight, y_groups, y_leftbag, y_rest, y_walk

In [23]:
X_test = np.concatenate((X_walk[-400:], X_groups[-200:], new_groups[-300:], X_fight[-200:], new_fight[-300:], X_leftbag[-500:], new_leftbag[-800:], X_rest[-249:]),axis=0)
y_test = np.concatenate((y_walk[-400:], y_groups[-200:], y_groups[-300:], y_fight[-200:], y_fight[-300:], y_leftbag[-500:], y_leftbag[-800:], y_rest[-249:]),axis=0)

X_test = X_test[:,10:285,30:335]
X_test = np.array([cv2.resize(i, (224,224)) for i in X_test[:,:]])

#X = np.array([cv2.cvtColor(i, cv2.COLOR_BGR2GRAY) for i in X[:]])   # TO GREYSCALE
#X = X.reshape(30, 90, 224, 224, 1)       # TO GREYSCALE

X_test = np.array([i for idx,i in enumerate(X_test[:]) if idx %3 != 0])
y_test = np.array([i for idx,i in enumerate(y_test[:]) if idx %3 != 0])


#For 1 classification
y_test[y_test == 2] = 1  

############## One hot encoding for 3 class
"""
onehot_encoder = OneHotEncoder(sparse=False)
y = onehot_encoder.fit_transform(y)
y = y.astype(np.int8)
y = y.reshape(88, 40, 3)
X = X.reshape(88, 40 ,224, 224, 3)"""

X_test = X_test[6:]
y_test = y_test[6:]

X_test = X_test.reshape(49, 40, 224, 224, 3)
y_test = y_test.reshape(49, 40, 1)

del X_fight, X_groups, X_leftbag, X_rest, X_walk, new_fight, new_leftbag, new_groups
del y_fight, y_groups, y_leftbag, y_rest, y_walk